In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import warnings
import pandas as pd
import numpy as np
warnings.filterwarnings(action='ignore')

# 데이터 불러오기

In [2]:
import pandas as pd

df = pd.read_csv("./volleyball_raw.csv", encoding='utf8')
df

,경기번호,팀명,경기날짜,결과,No.,이름,출전세트_1set,출전세트_2set,출전세트_3set,출전세트_4set,...,블로킹_시도,블로킹_성공,블로킹_유효블락,블로킹_실패,블로킹_범실,블로킹_세트당,블로킹_점유율,블로킹_어시스트,벌칙_벌칙,범실_범실
0,1,IBK기업은행,2017-10-14,0,19,메디 (L),O,O,O,O,...,20,3,5,6,1,0.6,21.28,1,0,7
1,1,IBK기업은행,2017-10-14,0,7,고예림 (L),O,O,O,O,...,10,0,7,3,0,0.0,10.64,0,0,6
2,1,IBK기업은행,2017-10-14,0,4,김희진 (R),O,O,O,O,...,14,4,6,3,0,0.8,14.89,1,0,6
3,1,IBK기업은행,2017-10-14,0,17,김미연 (L),O,O,O,O,...,12,2,2,7,0,0.4,12.77,3,0,1
4,1,IBK기업은행,2017-10-14,0,11,김수지 (C),O,O,O,O,...,28,1,13,9,0,0.2,29.79,2,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10992,474,KGC인삼공사,2022-02-28,0,14,김혜원 (S),O,NaN,2,NaN,...,2,0,1,1,0,0.0,4.44,0,0,0
10993,474,KGC인삼공사,2022-02-28,0,8,나현수 (C),NaN,17,12,NaN,...,1,0,0,1,0,0.0,2.22,0,0,2
10994,474,KGC인삼공사,2022-02-28,0,5,노란 (Li),LI,NaN,NaN,NaN,...,0,0,0,0,0,0.0,0.00,0,0,0
10995,474,KGC인삼공사,2022-02-28,0,15,이선우 (L),NaN,2,1,NaN,...,0,0,0,0,0,0.0,0.00,0,0,0


# 데이터 전처리

## 홈_어웨이 컬럼 추가

In [3]:
home=[]
away=[]
for i in range(1, len(df['경기번호'].unique())+1):
    home.append([i,df[df['경기번호']==i]['팀명'].unique()[0]])
    away.append([i,df[df['경기번호']==i]['팀명'].unique()[1]])
    
df["홈_어웨이"]=np.nan

for i in range(len(df['경기번호'].unique())):
    home_idx = df[(df["경기번호"]==home[i][0]) & (df['팀명']==home[i][1])].index
    away_idx = df[(df["경기번호"]==away[i][0]) & (df['팀명']==away[i][1])].index
    df.loc[home_idx,'홈_어웨이']=1
    df.loc[away_idx,'홈_어웨이']=0
df['홈_어웨이']=df["홈_어웨이"].astype("int64")

경기별_수치 = df.groupby(["경기번호", "팀명", "결과"]).sum()
경기별_수치=경기별_수치.reset_index().reset_index()
경기별_수치["홈_어웨이"]=경기별_수치["홈_어웨이"].apply(lambda x : 1 if x!=0 else 0)
경기별_수치

,index,경기번호,팀명,결과,No.,득점_득점,공격종합_시도,공격종합_성공,공격종합_공격차단,공격종합_범실,...,블로킹_성공,블로킹_유효블락,블로킹_실패,블로킹_범실,블로킹_세트당,블로킹_점유율,블로킹_어시스트,벌칙_벌칙,범실_범실,홈_어웨이
0,0,1,IBK기업은행,0,89,69,179,51,15,10,...,11,37,30,2,2.25,100.01,8,0,25,1
1,1,1,흥국생명,1,127,83,193,63,11,15,...,15,38,12,0,3.30,100.02,10,0,27,0
2,2,2,KGC인삼공사,0,94,79,205,67,20,7,...,7,31,22,2,1.40,99.99,5,0,14,1
3,3,2,현대건설,1,102,102,198,77,7,12,...,20,33,30,3,4.00,100.01,17,0,29,0
4,4,3,GS칼텍스,1,138,88,175,72,9,9,...,7,28,25,0,1.45,100.00,6,0,25,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,943,472,흥국생명,0,111,38,108,32,8,8,...,3,16,18,2,0.99,99.99,3,0,15,0
944,944,473,페퍼저축은행,0,116,44,123,36,10,7,...,4,16,13,0,1.33,100.01,3,0,12,0
945,945,473,한국도로공사,1,124,63,109,49,4,3,...,10,21,19,0,3.33,100.01,9,0,11,1
946,946,474,GS칼텍스,1,164,67,91,52,8,0,...,10,21,17,1,3.34,99.98,8,0,7,1


## 상대팀 컬럼 추가

In [4]:
home_index = 경기별_수치[경기별_수치['index']/2+1==경기별_수치['경기번호']].index
away_index = 경기별_수치[경기별_수치['index']/2+1!=경기별_수치['경기번호']].index
경기별_수치['상대팀'] = 경기별_수치['팀명']
경기별_수치.loc[home_index,'상대팀']=경기별_수치.loc[away_index,'팀명'].values
경기별_수치.loc[away_index,'상대팀']=경기별_수치.loc[home_index,'팀명'].values
경기별_수치

,index,경기번호,팀명,결과,No.,득점_득점,공격종합_시도,공격종합_성공,공격종합_공격차단,공격종합_범실,...,블로킹_유효블락,블로킹_실패,블로킹_범실,블로킹_세트당,블로킹_점유율,블로킹_어시스트,벌칙_벌칙,범실_범실,홈_어웨이,상대팀
0,0,1,IBK기업은행,0,89,69,179,51,15,10,...,37,30,2,2.25,100.01,8,0,25,1,흥국생명
1,1,1,흥국생명,1,127,83,193,63,11,15,...,38,12,0,3.30,100.02,10,0,27,0,IBK기업은행
2,2,2,KGC인삼공사,0,94,79,205,67,20,7,...,31,22,2,1.40,99.99,5,0,14,1,현대건설
3,3,2,현대건설,1,102,102,198,77,7,12,...,33,30,3,4.00,100.01,17,0,29,0,KGC인삼공사
4,4,3,GS칼텍스,1,138,88,175,72,9,9,...,28,25,0,1.45,100.00,6,0,25,0,한국도로공사
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,943,472,흥국생명,0,111,38,108,32,8,8,...,16,18,2,0.99,99.99,3,0,15,0,IBK기업은행
944,944,473,페퍼저축은행,0,116,44,123,36,10,7,...,16,13,0,1.33,100.01,3,0,12,0,한국도로공사
945,945,473,한국도로공사,1,124,63,109,49,4,3,...,21,19,0,3.33,100.01,9,0,11,1,페퍼저축은행
946,946,474,GS칼텍스,1,164,67,91,52,8,0,...,21,17,1,3.34,99.98,8,0,7,1,KGC인삼공사


## 팀명을 팀코드로 변환

In [5]:
a = ['GS칼텍스', '흥국생명', '한국도로공사', "IBK기업은행", "KGC인삼공사", "현대건설", "페퍼저축은행"]
b=list(range(0,7))
dic = dict(zip(a,b))
dic
경기별_수치['팀코드']= 경기별_수치['팀명'].replace(dic)

경기별_수치

,index,경기번호,팀명,결과,No.,득점_득점,공격종합_시도,공격종합_성공,공격종합_공격차단,공격종합_범실,...,블로킹_실패,블로킹_범실,블로킹_세트당,블로킹_점유율,블로킹_어시스트,벌칙_벌칙,범실_범실,홈_어웨이,상대팀,팀코드
0,0,1,IBK기업은행,0,89,69,179,51,15,10,...,30,2,2.25,100.01,8,0,25,1,흥국생명,3
1,1,1,흥국생명,1,127,83,193,63,11,15,...,12,0,3.30,100.02,10,0,27,0,IBK기업은행,1
2,2,2,KGC인삼공사,0,94,79,205,67,20,7,...,22,2,1.40,99.99,5,0,14,1,현대건설,4
3,3,2,현대건설,1,102,102,198,77,7,12,...,30,3,4.00,100.01,17,0,29,0,KGC인삼공사,5
4,4,3,GS칼텍스,1,138,88,175,72,9,9,...,25,0,1.45,100.00,6,0,25,0,한국도로공사,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,943,472,흥국생명,0,111,38,108,32,8,8,...,18,2,0.99,99.99,3,0,15,0,IBK기업은행,1
944,944,473,페퍼저축은행,0,116,44,123,36,10,7,...,13,0,1.33,100.01,3,0,12,0,한국도로공사,6
945,945,473,한국도로공사,1,124,63,109,49,4,3,...,19,0,3.33,100.01,9,0,11,1,페퍼저축은행,2
946,946,474,GS칼텍스,1,164,67,91,52,8,0,...,17,1,3.34,99.98,8,0,7,1,KGC인삼공사,0


## 각 수치별 비율 계산

In [6]:
경기별_수치['공격종합_성공률'] = round(경기별_수치["공격종합_성공"]/경기별_수치['공격종합_시도'], 2)
경기별_수치.drop("공격종합_점유율",axis=1, inplace=True)
경기별_수치['오픈_성공률'] = round(경기별_수치["오픈_성공"]/경기별_수치['오픈_시도'], 2)
경기별_수치.drop("오픈_점유율",axis=1, inplace=True)
경기별_수치['시간차_성공률'] = round(경기별_수치["시간차_성공"]/경기별_수치['시간차_시도'], 2)
경기별_수치.drop("시간차_점유율",axis=1, inplace=True)
경기별_수치['이동_성공률'] = round(경기별_수치["이동_성공"]/경기별_수치['이동_시도'], 2)
경기별_수치.drop("이동_점유율",axis=1, inplace=True)
경기별_수치['후위_성공률'] = round(경기별_수치["후위_성공"]/경기별_수치['후위_시도'], 2)
경기별_수치.drop("후위_점유율",axis=1, inplace=True)
경기별_수치['속공_성공률'] = round(경기별_수치["속공_성공"]/경기별_수치['속공_시도'], 2)
경기별_수치.drop("속공_점유율",axis=1, inplace=True)
경기별_수치['퀵오픈_성공률'] = round(경기별_수치["퀵오픈_성공"]/경기별_수치['퀵오픈_시도'], 2)
경기별_수치.drop("퀵오픈_점유율",axis=1, inplace=True)
경기별_수치['서브_성공률'] = round(경기별_수치["서브_성공"]/경기별_수치['서브_시도'], 2)
경기별_수치.drop("서브_점유율",axis=1, inplace=True)
경기별_수치['디그_성공률'] = round(경기별_수치["디그_성공"]/경기별_수치['디그_시도'], 2)
경기별_수치.drop("디그_점유율",axis=1, inplace=True)
경기별_수치['세트_성공률'] = round(경기별_수치["세트_성공"]/경기별_수치['세트_시도'], 2)
경기별_수치.drop("세트_점유율",axis=1, inplace=True)
경기별_수치['리시브_정확도'] = round(경기별_수치["리시브_정확"]/경기별_수치['리시브_시도'], 2)
경기별_수치.drop("리시브_점유율",axis=1, inplace=True)
경기별_수치['블로킹_성공률'] = round(경기별_수치["블로킹_성공"]/경기별_수치['블로킹_시도'], 2)
경기별_수치.drop("블로킹_점유율",axis=1, inplace=True)

경기별_공격 = 경기별_수치[["경기번호","팀명","결과","공격종합_시도","공격종합_성공"]]
경기별_공격["공격시도중_오픈비율"] = 경기별_수치["오픈_시도"]/경기별_수치["공격종합_시도"]
경기별_공격["공격성공중_오픈비율"] = 경기별_수치["오픈_성공"]/경기별_수치["공격종합_성공"]
경기별_공격["공격시도중_시간차비율"] = 경기별_수치["시간차_시도"]/경기별_수치["공격종합_시도"]
경기별_공격["공격성공중_시간차비율"] = 경기별_수치["시간차_성공"]/경기별_수치["공격종합_성공"]
경기별_공격["공격시도중_이동시율"] = 경기별_수치["이동_시도"]/경기별_수치["공격종합_시도"]
경기별_공격["공격성공중_이동성율"] = 경기별_수치["이동_성공"]/경기별_수치["공격종합_성공"]
경기별_공격["공격시도중_후위시율"] = 경기별_수치["후위_시도"]/경기별_수치["공격종합_시도"]
경기별_공격["공격성공중_후위성율"] = 경기별_수치["후위_성공"]/경기별_수치["공격종합_성공"]
경기별_공격["공격시도중_속공시율"] = 경기별_수치["속공_시도"]/경기별_수치["공격종합_시도"]
경기별_공격["공격성공중_속공성율"] = 경기별_수치["속공_성공"]/경기별_수치["공격종합_성공"]
경기별_공격["공격시도중_퀵오픈비율"] = 경기별_수치["퀵오픈_시도"]/경기별_수치["공격종합_시도"]
경기별_공격["공격성공중_퀵오픈비율"] = 경기별_수치["퀵오픈_성공"]/경기별_수치["공격종합_성공"]

경기별_수치

,index,경기번호,팀명,결과,No.,득점_득점,공격종합_시도,공격종합_성공,공격종합_공격차단,공격종합_범실,...,블로킹_어시스트,벌칙_벌칙,범실_범실,홈_어웨이,상대팀,팀코드,디그_성공률,세트_성공률,리시브_정확도,블로킹_성공률
0,0,1,IBK기업은행,0,89,69,179,51,15,10,...,8,0,25,1,흥국생명,3,0.86,0.27,0.40,0.12
1,1,1,흥국생명,1,127,83,193,63,11,15,...,10,0,27,0,IBK기업은행,1,0.86,0.31,0.51,0.21
2,2,2,KGC인삼공사,0,94,79,205,67,20,7,...,5,0,14,1,현대건설,4,0.81,0.30,0.48,0.09
3,3,2,현대건설,1,102,102,198,77,7,12,...,17,0,29,0,KGC인삼공사,5,0.89,0.36,0.45,0.19
4,4,3,GS칼텍스,1,138,88,175,72,9,9,...,6,0,25,0,한국도로공사,0,0.84,0.38,0.44,0.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,943,472,흥국생명,0,111,38,108,32,8,8,...,3,0,15,0,IBK기업은행,1,0.81,0.27,0.25,0.06
944,944,473,페퍼저축은행,0,116,44,123,36,10,7,...,3,0,12,0,한국도로공사,6,0.84,0.27,0.37,0.09
945,945,473,한국도로공사,1,124,63,109,49,4,3,...,9,0,11,1,페퍼저축은행,2,0.90,0.43,0.59,0.14
946,946,474,GS칼텍스,1,164,67,91,52,8,0,...,8,0,7,1,KGC인삼공사,0,0.82,0.52,0.48,0.17


## 총 득점 점수 계산

In [7]:
# 총 점수 계산 (득점 + 상대팀 범실)

tmp = 경기별_수치.copy()
home_index = 경기별_수치[경기별_수치['index']/2+1==경기별_수치['경기번호']].index
away_index = 경기별_수치[경기별_수치['index']/2+1!=경기별_수치['경기번호']].index
경기별_수치['총_점수'] = 경기별_수치['득점_득점']

경기별_수치.loc[home_index,'총_점수']+=tmp.loc[away_index, "범실_범실"].values
경기별_수치.loc[away_index,'총_점수']+=tmp.loc[home_index, "범실_범실"].values
경기별_수치

,index,경기번호,팀명,결과,No.,득점_득점,공격종합_시도,공격종합_성공,공격종합_공격차단,공격종합_범실,...,벌칙_벌칙,범실_범실,홈_어웨이,상대팀,팀코드,디그_성공률,세트_성공률,리시브_정확도,블로킹_성공률,총_점수
0,0,1,IBK기업은행,0,89,69,179,51,15,10,...,0,25,1,흥국생명,3,0.86,0.27,0.40,0.12,96
1,1,1,흥국생명,1,127,83,193,63,11,15,...,0,27,0,IBK기업은행,1,0.86,0.31,0.51,0.21,108
2,2,2,KGC인삼공사,0,94,79,205,67,20,7,...,0,14,1,현대건설,4,0.81,0.30,0.48,0.09,108
3,3,2,현대건설,1,102,102,198,77,7,12,...,0,29,0,KGC인삼공사,5,0.89,0.36,0.45,0.19,116
4,4,3,GS칼텍스,1,138,88,175,72,9,9,...,0,25,0,한국도로공사,0,0.84,0.38,0.44,0.10,110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,943,472,흥국생명,0,111,38,108,32,8,8,...,0,15,0,IBK기업은행,1,0.81,0.27,0.25,0.06,53
944,944,473,페퍼저축은행,0,116,44,123,36,10,7,...,0,12,0,한국도로공사,6,0.84,0.27,0.37,0.09,55
945,945,473,한국도로공사,1,124,63,109,49,4,3,...,0,11,1,페퍼저축은행,2,0.90,0.43,0.59,0.14,75
946,946,474,GS칼텍스,1,164,67,91,52,8,0,...,0,7,1,KGC인삼공사,0,0.82,0.52,0.48,0.17,78


## 실점 계산

In [8]:
# 실점 컬럼 추가 (본인팀 실점 = 상대팀 총 점수)
home_index = 경기별_수치[경기별_수치['index']/2+1==경기별_수치['경기번호']].index
away_index = 경기별_수치[경기별_수치['index']/2+1!=경기별_수치['경기번호']].index
경기별_수치['실점'] = np.nan

경기별_수치.loc[home_index,'실점']=경기별_수치.loc[away_index, "총_점수"].values
경기별_수치.loc[away_index,'실점']=경기별_수치.loc[home_index, "총_점수"].values
경기별_수치 = 경기별_수치.astype({'실점':"int64"})
경기별_수치


,index,경기번호,팀명,결과,No.,득점_득점,공격종합_시도,공격종합_성공,공격종합_공격차단,공격종합_범실,...,범실_범실,홈_어웨이,상대팀,팀코드,디그_성공률,세트_성공률,리시브_정확도,블로킹_성공률,총_점수,실점
0,0,1,IBK기업은행,0,89,69,179,51,15,10,...,25,1,흥국생명,3,0.86,0.27,0.40,0.12,96,108
1,1,1,흥국생명,1,127,83,193,63,11,15,...,27,0,IBK기업은행,1,0.86,0.31,0.51,0.21,108,96
2,2,2,KGC인삼공사,0,94,79,205,67,20,7,...,14,1,현대건설,4,0.81,0.30,0.48,0.09,108,116
3,3,2,현대건설,1,102,102,198,77,7,12,...,29,0,KGC인삼공사,5,0.89,0.36,0.45,0.19,116,108
4,4,3,GS칼텍스,1,138,88,175,72,9,9,...,25,0,한국도로공사,0,0.84,0.38,0.44,0.10,110,107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,943,472,흥국생명,0,111,38,108,32,8,8,...,15,0,IBK기업은행,1,0.81,0.27,0.25,0.06,53,75
944,944,473,페퍼저축은행,0,116,44,123,36,10,7,...,12,0,한국도로공사,6,0.84,0.27,0.37,0.09,55,75
945,945,473,한국도로공사,1,124,63,109,49,4,3,...,11,1,페퍼저축은행,2,0.90,0.43,0.59,0.14,75,55
946,946,474,GS칼텍스,1,164,67,91,52,8,0,...,7,1,KGC인삼공사,0,0.82,0.52,0.48,0.17,78,52


In [9]:
# 경기별_수치.to_csv('경기별_수치.csv')

## 홈/원정 경기장과 직전 세경기 결과에 따른 평균
- 각 컬럼에 따른 직전 세경기 결과의 평균 계산
- 초기 세 경기는 나머지 결과의 평균값을 대입

In [10]:
경기별_수치[경기별_수치['팀명']=="IBK기업은행"][["공격종합_성공률", "서브_성공률", "디그_성공률","세트_성공률","리시브_정확도","블로킹_성공률","범실_범실"]]

,공격종합_성공률,서브_성공률,디그_성공률,세트_성공률,리시브_정확도,블로킹_성공률,범실_범실
0,0.28,0.07,0.86,0.27,0.40,0.12,25
6,0.37,0.07,0.84,0.33,0.53,0.10,19
10,0.43,0.09,0.83,0.39,0.56,0.11,15
21,0.45,0.03,0.90,0.41,0.28,0.15,11
28,0.34,0.04,0.82,0.33,0.45,0.13,20
...,...,...,...,...,...,...,...
916,0.40,0.04,0.86,0.36,0.36,0.09,24
924,0.51,0.03,0.75,0.47,0.48,0.24,13
929,0.44,0.03,0.82,0.44,0.37,0.20,15
934,0.35,0.05,0.75,0.31,0.23,0.08,15


In [11]:
IBK = 경기별_수치[경기별_수치['팀명']=="IBK기업은행"][["홈_어웨이","공격종합_성공률", "서브_성공률", "디그_성공률","세트_성공률","리시브_정확도","블로킹_성공률","범실_범실"]]
GS = 경기별_수치[경기별_수치['팀명']=="GS칼텍스"][["홈_어웨이","공격종합_성공률", "서브_성공률", "디그_성공률","세트_성공률","리시브_정확도","블로킹_성공률","범실_범실"]]
흥국생명 = 경기별_수치[경기별_수치['팀명']=="흥국생명"][["홈_어웨이","공격종합_성공률", "서브_성공률", "디그_성공률","세트_성공률","리시브_정확도","블로킹_성공률","범실_범실"]]
현대건설 = 경기별_수치[경기별_수치['팀명']=="현대건설"][["홈_어웨이","공격종합_성공률", "서브_성공률", "디그_성공률","세트_성공률","리시브_정확도","블로킹_성공률","범실_범실"]]
한국도로공사 = 경기별_수치[경기별_수치['팀명']=="한국도로공사"][["홈_어웨이","공격종합_성공률", "서브_성공률", "디그_성공률","세트_성공률","리시브_정확도","블로킹_성공률","범실_범실"]]
KGC = 경기별_수치[경기별_수치['팀명']=="KGC인삼공사"][["홈_어웨이","공격종합_성공률", "서브_성공률", "디그_성공률","세트_성공률","리시브_정확도","블로킹_성공률","범실_범실"]]
페퍼 = 경기별_수치[경기별_수치['팀명']=="페퍼저축은행"][["홈_어웨이","공격종합_성공률", "서브_성공률", "디그_성공률","세트_성공률","리시브_정확도","블로킹_성공률","범실_범실"]]

In [12]:
흥국생명_홈 = 흥국생명[흥국생명['홈_어웨이']==1]
흥국생명_어웨이 = 흥국생명[흥국생명['홈_어웨이']==0]
IBK_홈 = IBK[IBK['홈_어웨이']==1]
IBK_어웨이 = IBK[IBK['홈_어웨이']==0]
GS_홈 = GS[GS['홈_어웨이']==1]
GS_어웨이 = GS[GS['홈_어웨이']==0]
현대건설_홈 = 현대건설[현대건설['홈_어웨이']==1]
현대건설_어웨이 = 현대건설[현대건설['홈_어웨이']==0]
한국도로공사_홈 = 한국도로공사[한국도로공사['홈_어웨이']==1]
한국도로공사_어웨이 = 한국도로공사[한국도로공사['홈_어웨이']==0]
KGC_홈 = KGC[KGC['홈_어웨이']==1]
KGC_어웨이 = KGC[KGC['홈_어웨이']==0]
페퍼_홈 = 페퍼[페퍼['홈_어웨이']==1]
페퍼_어웨이 = 페퍼[페퍼['홈_어웨이']==0]

In [13]:
팀_리스트 = [흥국생명_홈, 흥국생명_어웨이, IBK_홈, IBK_어웨이, GS_홈,GS_어웨이, 현대건설_홈,현대건설_어웨이, 한국도로공사_홈,한국도로공사_어웨이, KGC_홈,KGC_어웨이,페퍼_홈,페퍼_어웨이]

for i in 팀_리스트 :
    i['홈어웨이_공격성공률_평균']=np.nan
    i['홈어웨이_서브성공률_평균']=np.nan
    i['홈어웨이_디그성공률_평균']=np.nan
    i['홈어웨이_세트성공률_평균']=np.nan
    i['홈어웨이_리시브정확도_평균']=np.nan
    i['홈어웨이_블로킹성공률_평균']=np.nan
    i['홈어웨이_범실_평균']=np.nan
    
for i in 팀_리스트:
    for j in range(2, len(i)):
        i.iloc[j,8:] = i.iloc[j-2:j,1:8].mean().values

for i in 팀_리스트:
    i.iloc[0:2,8:]=i.iloc[2:,8:].mean()

df=팀_리스트[0].copy()
for i in 팀_리스트[1:]:
    df = pd.concat([df,i])
df = df.sort_index()
df.drop("홈_어웨이",axis=1, inplace=True)
경기별_수치 = pd.concat([경기별_수치, df], axis=1)

a = ['GS칼텍스', '흥국생명', '한국도로공사', "IBK기업은행", "KGC인삼공사", "현대건설", "페퍼저축은행"]
b=list(range(0,7))
dic = dict(zip(a,b))

경기별_수치['상대팀코드']= 경기별_수치['상대팀'].replace(dic)
경기별_수치

,index,경기번호,팀명,결과,No.,득점_득점,공격종합_시도,공격종합_성공,공격종합_공격차단,공격종합_범실,...,블로킹_성공률,범실_범실,홈어웨이_공격성공률_평균,홈어웨이_서브성공률_평균,홈어웨이_디그성공률_평균,홈어웨이_세트성공률_평균,홈어웨이_리시브정확도_평균,홈어웨이_블로킹성공률_평균,홈어웨이_범실_평균,상대팀코드
0,0,1,IBK기업은행,0,89,69,179,51,15,10,...,0.12,25,0.383117,0.055065,0.831558,0.356558,0.383182,0.129351,19.129870,1
1,1,1,흥국생명,1,127,83,193,63,11,15,...,0.21,27,0.364675,0.045000,0.830130,0.337273,0.409286,0.128636,17.915584,3
2,2,2,KGC인삼공사,0,94,79,205,67,20,7,...,0.09,14,0.370274,0.045753,0.832466,0.338356,0.410205,0.127123,18.328767,5
3,3,2,현대건설,1,102,102,198,77,7,12,...,0.19,29,0.387600,0.051000,0.837933,0.352200,0.392133,0.135200,19.766667,4
4,4,3,GS칼텍스,1,138,88,175,72,9,9,...,0.10,25,0.406039,0.048182,0.828377,0.376299,0.410714,0.134091,19.461039,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,943,472,흥국생명,0,111,38,108,32,8,8,...,0.06,15,0.355000,0.030000,0.825000,0.320000,0.340000,0.105000,17.000000,3
944,944,473,페퍼저축은행,0,116,44,123,36,10,7,...,0.09,12,0.370000,0.035000,0.735000,0.340000,0.325000,0.090000,14.000000,2
945,945,473,한국도로공사,1,124,63,109,49,4,3,...,0.14,11,0.375000,0.020000,0.835000,0.350000,0.350000,0.100000,12.500000,6
946,946,474,GS칼텍스,1,164,67,91,52,8,0,...,0.17,7,0.445000,0.090000,0.780000,0.425000,0.505000,0.140000,16.500000,4


## 직전 다섯경기 결과에 따른 평균 계산
- 각 컬럼에 따른 직전 다섯개의 경기 결과의 평균 계산
- 초기 다섯개의 경기는 나머지 결과의 평균값을 대입

In [14]:
import pandas as pd
df = pd.read_csv("여자배구_원정홈비율.csv")
df = df.drop(columns=['Unnamed: 0','index'])

df=df.reset_index()

# 직전 다섯경기 IBK기업은행 평균
data = pd.DataFrame()
팀 = df['팀명'].unique()

for i in 팀:
    temp = df[df['팀명'] == i][['index','팀명','경기번호','홈_어웨이', '결과', '범실_범실', '디그_성공률',
                                     '리시브_정확도','공격종합_성공률','블로킹_성공률', '세트_성공률','서브_성공률']]

    temp_cal = pd.DataFrame(index = range(len(temp['경기번호'])),columns=['index','팀명','범실_평균_다섯', '디그_평균_다섯',
                                     '리시브_평균_다섯','공격종합_평균_다섯','블로킹_평균_다섯', '세트_평균_다섯','서브_평균_다섯'])

    for i in range(5,len(temp['경기번호'])):
        temp_cal.iloc[i]['팀명'] = temp.iloc[i]['팀명']
        temp_cal.iloc[i]['index'] = temp.iloc[i]['index']
        temp_cal.iloc[i]['범실_평균_다섯'] = temp.iloc[i-5:i]['범실_범실'].mean()
        temp_cal.iloc[i]['디그_평균_다섯'] = temp.iloc[i-5:i]['디그_성공률'].mean()
        temp_cal.iloc[i]['리시브_평균_다섯'] = temp.iloc[i-5:i]['리시브_정확도'].mean()
        temp_cal.iloc[i]['공격종합_평균_다섯'] = temp.iloc[i-5:i]['공격종합_성공률'].mean()
        temp_cal.iloc[i]['블로킹_평균_다섯'] = temp.iloc[i-5:i]['블로킹_성공률'].mean()
        temp_cal.iloc[i]['세트_평균_다섯'] = temp.iloc[i-5:i]['세트_성공률'].mean()
        temp_cal.iloc[i]['서브_평균_다섯'] = temp.iloc[i-5:i]['서브_성공률'].mean()
    for i in range(0,5):
        temp_cal.iloc[i]['팀명'] = temp.iloc[i]['팀명']
        temp_cal.iloc[i]['index'] = temp.iloc[i]['index']
        temp_cal.iloc[i]['범실_평균_다섯'] = temp_cal['범실_평균_다섯'].mean()
        temp_cal.iloc[i]['디그_평균_다섯'] = temp_cal['디그_평균_다섯'].mean()
        temp_cal.iloc[i]['리시브_평균_다섯'] = temp_cal['리시브_평균_다섯'].mean()
        temp_cal.iloc[i]['공격종합_평균_다섯'] = temp_cal['공격종합_평균_다섯'].mean()
        temp_cal.iloc[i]['블로킹_평균_다섯'] = temp_cal['블로킹_평균_다섯'].mean()
        temp_cal.iloc[i]['세트_평균_다섯'] = temp_cal['세트_평균_다섯'].mean()
        temp_cal.iloc[i]['서브_평균_다섯'] = temp_cal['서브_평균_다섯'].mean()
    data = pd.concat([data,temp_cal])

data = data.sort_values(by=['index'], axis=0)
data

,index,팀명,범실_평균_다섯,디그_평균_다섯,리시브_평균_다섯,공격종합_평균_다섯,블로킹_평균_다섯,세트_평균_다섯,서브_평균_다섯
0,0,IBK기업은행,19.26755,0.830013,0.378954,0.373139,0.124318,0.347192,0.049483
0,1,흥국생명,18.210526,0.835789,0.392145,0.366355,0.125658,0.339421,0.048408
0,2,KGC인삼공사,18.506383,0.828156,0.401773,0.365348,0.123277,0.332156,0.04427
0,3,현대건설,19.718621,0.837448,0.397945,0.392731,0.140152,0.358676,0.050566
0,4,GS칼텍스,18.908108,0.835608,0.419419,0.406662,0.136446,0.376486,0.053095
...,...,...,...,...,...,...,...,...,...
156,943,흥국생명,17.2,0.82,0.342,0.338,0.118,0.308,0.044
29,944,페퍼저축은행,18.2,0.768,0.316,0.374,0.128,0.348,0.04
155,945,한국도로공사,16.2,0.844,0.434,0.362,0.094,0.336,0.026
152,946,GS칼텍스,15.8,0.82,0.5,0.44,0.13,0.408,0.062


In [15]:
# data.to_csv('직전다섯경기평균.csv')

## 홈/원정 경기장과 직전 세경기 결과에 따른 평균 계산
- 각 컬럼에 따른 직전 세경기 결과의 평균 계산
- 초기 세 경기는 나머지 결과의 평균값을 대입

In [16]:
# 직전 세경기 상대전적
data2 = pd.DataFrame()
팀 = df['팀명'].unique()

for i in 팀:
    temp = df[df['팀명'] == i][['index','팀명','상대팀','범실_범실', '디그_성공률',
                                     '리시브_정확도','공격종합_성공률','블로킹_성공률', '세트_성공률','서브_성공률']]
    for j in 팀:
        if i == j: continue
        temp2 = temp[temp['상대팀']==j].copy()
        temp_cal = pd.DataFrame(index = range(len(temp2['index'])),columns=['index','팀명','범실_상대전적', '디그_상대전적',
                                         '리시브_상대전적','공격종합_상대전적','블로킹_상대전적', '세트_상대전적','서브_상대전적'])

        for k in range(3,len(temp2['index'])):
            temp_cal.iloc[k]['index'] = temp2.iloc[k]['index']
            temp_cal.iloc[k]['팀명'] = temp2.iloc[k]['팀명']
            temp_cal.iloc[k]['범실_상대전적'] = temp.iloc[k-3:k]['범실_범실'].mean()
            temp_cal.iloc[k]['디그_상대전적'] = temp.iloc[k-3:k]['디그_성공률'].mean()
            temp_cal.iloc[k]['리시브_상대전적'] = temp.iloc[k-3:k]['리시브_정확도'].mean()
            temp_cal.iloc[k]['공격종합_상대전적'] = temp.iloc[k-3:k]['공격종합_성공률'].mean()
            temp_cal.iloc[k]['블로킹_상대전적'] = temp.iloc[k-3:k]['블로킹_성공률'].mean()
            temp_cal.iloc[k]['세트_상대전적'] = temp.iloc[k-3:k]['세트_성공률'].mean()
            temp_cal.iloc[k]['서브_상대전적'] = temp.iloc[k-3:k]['서브_성공률'].mean()
        for k in range(0,3):
            temp_cal.iloc[k]['index'] = temp2.iloc[k]['index']
            temp_cal.iloc[k]['팀명'] = temp2.iloc[k]['팀명']
            temp_cal.iloc[k]['범실_상대전적'] = temp_cal['범실_상대전적'].mean()
            temp_cal.iloc[k]['디그_상대전적'] = temp_cal['디그_상대전적'].mean()
            temp_cal.iloc[k]['리시브_상대전적'] = temp_cal['리시브_상대전적'].mean()
            temp_cal.iloc[k]['공격종합_상대전적'] = temp_cal['공격종합_상대전적'].mean()
            temp_cal.iloc[k]['블로킹_상대전적'] = temp_cal['블로킹_상대전적'].mean()
            temp_cal.iloc[k]['세트_상대전적'] = temp_cal['세트_상대전적'].mean()
            temp_cal.iloc[k]['서브_상대전적'] = temp_cal['서브_상대전적'].mean()
        data2 = pd.concat([data2,temp_cal])

data2 = data2.sort_values(by=['index'], axis=0)
data2

,index,팀명,범실_상대전적,디그_상대전적,리시브_상대전적,공격종합_상대전적,블로킹_상대전적,세트_상대전적,서브_상대전적
0,0,IBK기업은행,18.617284,0.853457,0.410494,0.397284,0.134691,0.371975,0.059259
0,1,흥국생명,19.962963,0.840864,0.436049,0.337778,0.111111,0.31358,0.055556
0,2,KGC인삼공사,17.653846,0.838846,0.395256,0.342179,0.129744,0.30859,0.042051
0,3,현대건설,17.25641,0.842949,0.422051,0.391795,0.169615,0.360385,0.059487
0,4,GS칼텍스,19.908046,0.834713,0.434368,0.397701,0.118046,0.370575,0.053678
...,...,...,...,...,...,...,...,...,...
29,943,흥국생명,20.0,0.843333,0.46,0.346667,0.103333,0.326667,0.05
4,944,페퍼저축은행,22.666667,0.733333,0.35,0.386667,0.13,0.356667,0.023333
4,945,한국도로공사,21.0,0.883333,0.503333,0.36,0.18,0.326667,0.066667
28,946,GS칼텍스,17.333333,0.8,0.426667,0.423333,0.163333,0.4,0.056667


In [17]:
# data2.to_csv('직전세경기상대전적.csv')

In [18]:
df5 = pd.read_csv("직전다섯경기평균.csv")
경기별_수치 = pd.concat([경기별_수치, df5.iloc[:,3:]],axis=1)

a = 경기별_수치[["결과", "홈_어웨이","팀코드","상대팀코드"]]

b = pd.concat([경기별_수치.iloc[:,83:90], 경기별_수치.iloc[:,91:]],axis=1)
데이터 = pd.concat([a,b],axis=1)
데이터

,결과,홈_어웨이,팀코드,상대팀코드,홈어웨이_공격성공률_평균,홈어웨이_서브성공률_평균,홈어웨이_디그성공률_평균,홈어웨이_세트성공률_평균,홈어웨이_리시브정확도_평균,홈어웨이_블로킹성공률_평균,홈어웨이_범실_평균,범실_평균_다섯,디그_평균_다섯,리시브_평균_다섯,공격종합_평균_다섯,블로킹_평균_다섯,세트_평균_다섯,서브_평균_다섯
0,0,1,3,1,0.383117,0.055065,0.831558,0.356558,0.383182,0.129351,19.129870,19.267550,0.830013,0.378954,0.373139,0.124318,0.347192,0.049483
1,1,0,1,3,0.364675,0.045000,0.830130,0.337273,0.409286,0.128636,17.915584,18.210526,0.835789,0.392145,0.366355,0.125658,0.339421,0.048408
2,0,1,4,5,0.370274,0.045753,0.832466,0.338356,0.410205,0.127123,18.328767,18.506383,0.828156,0.401773,0.365348,0.123277,0.332156,0.044270
3,1,0,5,4,0.387600,0.051000,0.837933,0.352200,0.392133,0.135200,19.766667,19.718621,0.837448,0.397945,0.392731,0.140152,0.358676,0.050566
4,1,0,0,2,0.406039,0.048182,0.828377,0.376299,0.410714,0.134091,19.461039,18.908108,0.835608,0.419419,0.406662,0.136446,0.376486,0.053095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,0,0,1,3,0.355000,0.030000,0.825000,0.320000,0.340000,0.105000,17.000000,17.200000,0.820000,0.342000,0.338000,0.118000,0.308000,0.044000
944,0,0,6,2,0.370000,0.035000,0.735000,0.340000,0.325000,0.090000,14.000000,18.200000,0.768000,0.316000,0.374000,0.128000,0.348000,0.040000
945,1,1,2,6,0.375000,0.020000,0.835000,0.350000,0.350000,0.100000,12.500000,16.200000,0.844000,0.434000,0.362000,0.094000,0.336000,0.026000
946,1,1,0,4,0.445000,0.090000,0.780000,0.425000,0.505000,0.140000,16.500000,15.800000,0.820000,0.500000,0.440000,0.130000,0.408000,0.062000


In [19]:
df3 = pd.read_csv("./직전세경기상대전적.csv")
데이터 = pd.concat([데이터, df3.iloc[:,3:]],axis=1)
데이터

,결과,홈_어웨이,팀코드,상대팀코드,홈어웨이_공격성공률_평균,홈어웨이_서브성공률_평균,홈어웨이_디그성공률_평균,홈어웨이_세트성공률_평균,홈어웨이_리시브정확도_평균,홈어웨이_블로킹성공률_평균,...,블로킹_평균_다섯,세트_평균_다섯,서브_평균_다섯,범실_상대전적,디그_상대전적,리시브_상대전적,공격종합_상대전적,블로킹_상대전적,세트_상대전적,서브_상대전적
0,0,1,3,1,0.383117,0.055065,0.831558,0.356558,0.383182,0.129351,...,0.124318,0.347192,0.049483,18.617284,0.853457,0.410494,0.397284,0.134691,0.371975,0.059259
1,1,0,1,3,0.364675,0.045000,0.830130,0.337273,0.409286,0.128636,...,0.125658,0.339421,0.048408,19.962963,0.840864,0.436049,0.337778,0.111111,0.313580,0.055556
2,0,1,4,5,0.370274,0.045753,0.832466,0.338356,0.410205,0.127123,...,0.123277,0.332156,0.044270,17.653846,0.838846,0.395256,0.342179,0.129744,0.308590,0.042051
3,1,0,5,4,0.387600,0.051000,0.837933,0.352200,0.392133,0.135200,...,0.140152,0.358676,0.050566,17.256410,0.842949,0.422051,0.391795,0.169615,0.360385,0.059487
4,1,0,0,2,0.406039,0.048182,0.828377,0.376299,0.410714,0.134091,...,0.136446,0.376486,0.053095,19.908046,0.834713,0.434368,0.397701,0.118046,0.370575,0.053678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,0,0,1,3,0.355000,0.030000,0.825000,0.320000,0.340000,0.105000,...,0.118000,0.308000,0.044000,20.000000,0.843333,0.460000,0.346667,0.103333,0.326667,0.050000
944,0,0,6,2,0.370000,0.035000,0.735000,0.340000,0.325000,0.090000,...,0.128000,0.348000,0.040000,22.666667,0.733333,0.350000,0.386667,0.130000,0.356667,0.023333
945,1,1,2,6,0.375000,0.020000,0.835000,0.350000,0.350000,0.100000,...,0.094000,0.336000,0.026000,21.000000,0.883333,0.503333,0.360000,0.180000,0.326667,0.066667
946,1,1,0,4,0.445000,0.090000,0.780000,0.425000,0.505000,0.140000,...,0.130000,0.408000,0.062000,17.333333,0.800000,0.426667,0.423333,0.163333,0.400000,0.056667


## 최종 여자배구 전처리 데이터 생성

In [20]:
# 데이터.to_csv("모델링데이터.csv")